In [8]:
!pip install numpy
!pip install nltk
!pip install scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [120]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
import pickle

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

movie_data = load_files(r"./txt_sentoken")
X, y = movie_data.data, movie_data.target

print(X[502])
print(y[502])

def lemmatize(X):
    documents = []

    lemmatizer = WordNetLemmatizer()

    for sen in range(0, len(X)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(X[sen]))
        
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        # Converting to Lowercase
        document = document.lower()
        
        # Lemmatization
        document = document.split()

        sentence = ' '.join(document)
        document = []
        
        for word, tag in pos_tag(word_tokenize(sentence)):
            wntag = tag[0].lower()
            wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
            lemma = lemmatizer.lemmatize(word, wntag) if wntag else word
            document.append(lemma)

        document = ' '.join(document)
        
        documents.append(document)

    return documents

documents = lemmatize(X)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joogps/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joogps/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joogps/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joogps/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


b'finding the courage to face life full-on is a difficult task . \nsummoning up the audacity to fall in love is a harder job . \nif you\'re a total jerk like melvin udall is , the odds are even more against you . \nmelvin ( jack nicholson ) is a successful romance author who is just the opposite of what you would expect someone in that profession to be . \na solitary man hidden away in his new york apartment with no human interaction and not even a hint of ever having been in a passionate relationship , he avoids the outside world as much as possible . \nhis obsessive-compulsive disorder rules his life as he uses several different bars of soap to wash his hands and refuses to step on a crack on the sidewalk . \n ( there\'s a great scene when he has to cross a floor made of mosaic tiles -- _very_small_ mosaic tiles . ) \nmelvin has some great lines . \nwhen his publisher\'s receptionist asks him how he writes women so well in his books , he responds " i think of men and i take away reas

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=4000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# print without truncating
# with np.printoptions(threshold=np.inf):
#     print(X_train)
#     print(y_train)

In [132]:
# from sklearn.naive_bayes import GaussianNB

# model = GaussianNB()

# model.fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, random_state=0)
model.fit(X, y)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [133]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[208   0]
 [  0 192]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       208
           1       1.00      1.00      1.00       192

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

1.0


In [140]:
sentences = ["movie is good", "movie is bad"]

lemmatized = lemmatize(sentences)

tokenized = vectorizer.transform(sentences).toarray()

#print without truncating
# with np.printoptions(threshold=np.inf):
#     print(tokenized)

print(model.predict(tokenized))

[1 0 1 0]
